# Filtre des réponses des utilisateurs pour proposer des festivals qui correspondent

**Ce qui nous intéresse dans cette partie est d'utiliser les réponses des utilisateurs pour déterminer des festivals qui correspondent aux attentes.**
<br>**Pour ce faire, nous utilisons les réponses provenant de question.py et envoyons les festivals dans app.py**

## I. Récupérer les informations

### I.1. Réponses de l'utilisateur

In [80]:
import json

# Charger les données utilisateur collectées
try:
    with open("user_data.json", "r") as json_file:
        user_data = json.load(json_file)
        print("Données utilisateur récupérées :", user_data)
except FileNotFoundError:
    print("Aucune donnée utilisateur n'a été collectée. Lancez 'questions.py' pour collecter les données.")

Données utilisateur récupérées : {'name': 'Neau Arthur', 'email': 'neau.arthur@sfr.fr', 'address': '14 Rue  Laurent Vibert 13090 Aix-en-Provence', 'coordinates': [43.533113, 5.43189], 'distance_max': 500, 'types': ['Musique'], 'genres musique': ['Musiques du monde'], 'genres spectacles vivants': [], 'genres cinéma et audiovisuel': [], 'genres arts visuels et numériques': [], 'genres livre et littérature': [], 'dates': ['2025-01-01', '2025-06-20'], 'budget': [20, 100], 'accessible': False}


In [6]:
user_data = {'name': 'Neau Arthur', 'email': 'neau.arthur@sfr.fr', 'address': '14 Rue Laurent Vibert, 13090 Aix-en-Provence, France', 'coordinates': [43.9352448339, 4.84071572505], 'distance_max': 500, 'types': ['Musique'], 'genres musique': ['Musiques du monde'], 'genres spectacles vivants': [], 'genres cinéma et audiovisuel': [], 'genres arts visuels et numériques': [], 'genres livre et littérature': [], 'dates': ['2025-01-01', '2025-06-20'], 'budget': [20, 100], 'accessible': False}


### I.2. Base de données des festivals

In [13]:
# Importer le fichier comme un module
from Nettoyage_base_donnees import df
df_a_filtrer = df
print(df_a_filtrer.head(20))


                                      Nom du festival Envergure territoriale  \
0                                    Festival andalou              Régionale   
1                            Festival Rêves d'enfants                    NaN   
3                                           Pharaonic                    NaN   
4                                     Sarcus Festival                    NaN   
5                                    Jazz in Fougères                    NaN   
6                                            L'aparté                    NaN   
7                    Art contemporain en milieu rural                    NaN   
8                        Les Musicales du Noirmoutier                    NaN   
10                                         Ai que Bom                    NaN   
11                                 Printemps des Rues         Intercommunale   
12                               Rhinoférock festival                    NaN   
13                             Festival 

## II. Filtrer les festivals

In [ ]:
from geopy.distance import geodesic
import pandas as pd

# Liste pour stocker les festivals correspondant
filtered_festivals = []

for _, row in df_a_filtrer.iterrows():
    # 1. Vérifier la distance
    if isinstance(row['Géocodage xy'], str):
        try:
            lat, lon = map(float, row['Géocodage xy'].split(','))
            distance = geodesic(user_data['coordinates'], (lat, lon)).km
            print(distance)
            if distance < user_data['distance_max']:
                filtered_festivals.append(row)
        except ValueError:
            continue  # Ignorer les lignes avec des coordonnées mal formatées
    else:
        continue
    print(filtered_festivals)

    # 2. Vérifier les dates (en ignorant l'année)
    # Correspondance entre les saisons et les plages de dates (mois et jours uniquement)
    seasons = {
    "Avant-saison (1er janvier - 20 juin)": ("01-01", "06-20"),
    "Saison (21 juin - 5 septembre)": ("06-21", "09-05"),
    "Après-saison (6 septembre - 31 décembre)": ("09-06", "12-31")
    }

    if row['Période principale de déroulement du festival'] in seasons:
        season_start, season_end = seasons[row['Période principale de déroulement du festival']]
    
        # Convertir les dates utilisateur (ignorer l'année)
        user_start = pd.to_datetime(user_data['dates'][0]).strftime("%m-%d")
        user_end = pd.to_datetime(user_data['dates'][1]).strftime("%m-%d")
    
        # Vérifier l'intersection entre les plages de dates
        # Convertir les dates de saison et utilisateur en plages de date
        season_range = pd.date_range(f"2024-{season_start}", f"2024-{season_end}")
        user_range = pd.date_range(f"2024-{user_start}", f"2024-{user_end}")

        # Vérifie si au moins un jour des plages utilisateur et saison se chevauchent
        if not any(day in season_range for day in user_range):
            continue
    else:
        continue


    # 3. Vérifier les types et sous-catégories
    category_match = False  # Indique si une catégorie correspond
    festival_categories = [cat.strip() for cat in row['Discipline dominante'].split(';')]  # Extraire les catégories du festival

    for category, type_col in zip(
        ['Musique', 'Spectacle vivant', 'Arts visuels et numériques', 'Cinéma et audiovisuel', 'Livre et littérature'],
        ['Nouvelles sous-catégories musique', 
         'Nouvelles sous-catégories spectacle vivant', 
         'Nouvelles sous-catégories arts visuels', 
         'Nouvelles sous-catégories cinéma et audiovisuel', 
         'Nouvelles sous-catégories livre et littérature']
    ):
        # Vérifier si la catégorie de l'utilisateur correspond à une catégorie du festival
        if category in user_data['types'] and category in festival_categories:
            subcategories = eval(row[type_col]) if isinstance(row[type_col], str) else []
            user_genres_key = f'genres {category.lower()}'
            user_subcategories = user_data.get(user_genres_key, [])
            
            # Vérifier si au moins une sous-catégorie correspond
            if any(subcategory in user_subcategories for subcategory in subcategories):
                category_match = True
                break  # Pas besoin de vérifier d'autres catégories

    # Si aucune catégorie ne correspond, on exclut le festival
    if not category_match:
        continue

    # Ajouter le festival filtré
    filtered_festivals.append(row)

# Convertir les résultats en DataFrame
filtered_df = pd.DataFrame(filtered_festivals)

# Afficher les résultats
print(filtered_df)


0.0
179.52501147341985
201.71624022940907
458.8292560138562
676.7037128147914
74.35443283130463
515.9579232713802
651.6842654201519
578.9479277531558
579.5254837698473
17.33153027840692
681.8879156860705
586.9261785538281
659.3414328271276
509.2095971307501
35.17549487817903
166.41877527844045
70.50672265548788
148.94220156778542
143.0213717916123
77.17170768335131
546.2370135581369
329.054425099721
581.8265801282989
583.340492776647
16.900738798926586
153.04460472208584
605.3456013406271
51.198158811218065
179.3797843249185
212.52773327212827
209.51034573960462
701.7019450561095
44.25884191607974
463.2821348433737
153.92067967209053
82.84562556313524
628.5249959431643
577.2413189502448
580.5126055062478
582.1496625282963
685.5507856285532
685.5507856285532
521.6996341819427
585.5301393685386
271.36277312819857
45.48542891032205
0.0
874.8201356439849
188.33812827769214
179.3797843249185
76.62104915083756
113.20515650903583
638.0456101586002
632.4140144971155
632.4140144971155
775.59402

## III. Classer et limiter les festivals

In [ ]:
# Coordonnées de référence
coordonnees = user_data['coordinates']

# Calculer la distance et ajouter une colonne
def calculate_distance(geo_string, reference_coords):
    try:
        lat, lon = map(float, geo_string.split(','))
        return geodesic(reference_coords, (lat, lon)).km
    except (ValueError, AttributeError):
        return None  # Retourne None en cas d'erreur de parsing

filtered_df['Distance'] = filtered_df['Géocodage xy'].apply(lambda geo: calculate_distance(geo, coordonnees))

print(filtered_df)

# Classement par distance
filtered_df_sorted = filtered_df.sort_values(by='Distance', ascending=True)

# On garde les 3 plus proches
df_end = filtered_df_sorted.head(3)

# Exporter df_end dans un fichier CSV
df_end.to_csv('df_end.csv', index=False)